In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

使用Vertex AI TensorBoard Profiler对配置文件模型进行培训性能分析

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tensorboard/tensorboard_profiler_custom_training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tensorboard/tensorboard_profiler_custom_training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/tensorboard/tensorboard_profiler_custom_training.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>

## 概述

Vertex AI TensorBoard 分析器允许您通过帮助您了解训练操作的资源消耗来监视和优化模型训练性能。本教程演示了如何启用Vertex AI TensorBoard分析器，以便您可以调试自定义训练作业的模型训练性能。

了解更多关于[Vertex AI TensorBoard 分析器](https://cloud.google.com/vertex-ai/docs/training/tensorboard-profiler)。

### 目标

在本教程中，您将学习如何为自定义训练作业启用Vertex AI TensorBoard分析器。

本教程使用以下Google Cloud AI服务：

- Vertex AI 训练
- Vertex AI TensorBoard

执行的步骤包括：

- 设置一个服务账号和一个Cloud Storage存储桶
- 创建一个TensorBoard实例
- 创建并运行一个启用了TensorBoard分析器的自定义训练作业
- 查看TensorBoard分析器仪表板以调试您的模型训练性能。

数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/overview)的[mnist数据集](https://www.tensorflow.org/datasets/catalog/mnist)。

### 费用

本教程使用Google Cloud的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成费用估算。

## 安装

安装以下必要的软件包以执行此笔记本。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform 

只有协作：取消注释以下单元格以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### 设置您的谷歌云项目

**无论您使用什么笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建一个账户时，您将获得$300的免费信用用于您的计算/存储成本。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

4. 如果您正在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面： [找到项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您也可以更改 Vertex AI 使用的 `REGION` 变量。了解更多关于 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type:"string"}

### 验证您的谷歌云账户

根据您的Jupyter环境，您可能需要手动认证。请按照下面的相关说明进行操作。

1. 顶点 AI 工作台
* 无需操作，因为您已经通过身份验证。

2. 本地 JupyterLab 实例，取消注释并运行:

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 设置服务账户和权限

服务账户用于创建自定义训练作业。如果您不想使用项目的计算引擎服务账户，请将SERVICE_ACCOUNT设置为另一个服务账户ID。您可以按照[说明](https://cloud.google.com/iam/docs/creating-managing-service-accounts#creating)创建服务账户。

In [ ]:
import sys

SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

In [ ]:
# Grant Cloud Storage permission.
! gcloud projects add-iam-policy-binding $PROJECT_ID \
        --member="serviceAccount:$SERVICE_ACCOUNT" \
        --role="roles/storage.admin" \
        --quiet

# Grant AI Platform permission.
! gcloud projects add-iam-policy-binding $PROJECT_ID \
        --member="serviceAccount:$SERVICE_ACCOUNT" \
        --role="roles/aiplatform.user" \
        --quiet

! gcloud projects get-iam-policy $PROJECT_ID \
        --filter=bindings.members:serviceAccount:$SERVICE_ACCOUNT

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶尚不存在：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

###导入库

In [ ]:
import os

from google.cloud import aiplatform

### 初始化用于Python的Vertex AI SDK

为您的项目和相应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

启用Artifact Registry API

首先，您必须为您的项目启用Artifact Registry API服务。

了解更多关于[启用服务](https://cloud.google.com/artifact-registry/docs/enable-service)。

In [ ]:
! gcloud services enable artifactregistry.googleapis.com --quiet

if os.getenv("IS_TESTING"):
    ! sudo apt-get update --yes && sudo apt-get --only-upgrade --yes install google-cloud-sdk-cloud-run-proxy google-cloud-sdk-harbourbridge google-cloud-sdk-cbt google-cloud-sdk-gke-gcloud-auth-plugin google-cloud-sdk-kpt google-cloud-sdk-local-extract google-cloud-sdk-minikube google-cloud-sdk-app-engine-java google-cloud-sdk-app-engine-go google-cloud-sdk-app-engine-python google-cloud-sdk-spanner-emulator google-cloud-sdk-bigtable-emulator google-cloud-sdk-nomos google-cloud-sdk-package-go-module google-cloud-sdk-firestore-emulator kubectl google-cloud-sdk-datastore-emulator google-cloud-sdk-app-engine-python-extras google-cloud-sdk-cloud-build-local google-cloud-sdk-kubectl-oidc google-cloud-sdk-anthos-auth google-cloud-sdk-app-engine-grpc google-cloud-sdk-pubsub-emulator google-cloud-sdk-datalab google-cloud-sdk-skaffold google-cloud-sdk google-cloud-sdk-terraform-tools google-cloud-sdk-config-connector
    ! gcloud components update --quiet

TensorBoard实例

在展示Vertex AI TensorBoard实验之前，必须先创建一个专门用于存储实验的区域化资源——Vertex AI TensorBoard实例。您可以在一个项目中创建多个实例。您可以使用命令 `gcloud ai tensorboards list` 来获取已存在的TensorBoard实例列表。

将您的TensorBoard实例显示名称设置为####

In [ ]:
TENSORBOARD_NAME = f"your-tensorboard-{PROJECT_ID}-unique"  # @param {type:"string"}

#### 创建一个TensorBoard实例

如果您还没有TensorBoard实例，请通过运行以下代码单元格来创建一个：

In [ ]:
tensorboard = aiplatform.Tensorboard.create(
    display_name=TENSORBOARD_NAME, project=PROJECT_ID, location=REGION
)

TENSORBOARD_INSTANCE_NAME = tensorboard.resource_name

print("TensorBoard instance name:", TENSORBOARD_INSTANCE_NAME)

## 训练模型

使用您的自定义训练代码训练模型时，选择以下选项之一：

- **预构建容器**：将您的自定义训练代码加载为 Python 包到 Google Cloud 的预构建容器镜像中。

- **自定义容器**：创建包含您的自定义训练代码的自定义容器镜像。

在本教程中，我们将使用自定义容器来训练一个自定义模型。

创建一个私有的Docker仓库

您的第一步是在Google Artifact Registry中创建自己的Docker仓库。

In [ ]:
DOCKER_REPOSITORY = f"my-docker-repo-{PROJECT_ID}-unique"

! gcloud artifacts repositories create {DOCKER_REPOSITORY} \
    --repository-format=docker \
    --location={REGION} \
    --description="Repository for TensorBoard Custom Training Job" \
    --quiet

! gcloud artifacts repositories list

### 配置身份验证到您的私有Docker仓库

在推送或拉取容器镜像之前，配置Docker使用`gcloud`命令行工具来对您区域的Artifact Registry进行身份验证请求。

In [ ]:
import sys

if not IS_COLAB:
    ! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

### 创建自定义容器镜像并推送到私人的Docker仓库

首先，您需要创建一个训练脚本文件和一个Docker文件。

创建一个文件夹用来存放所有的训练代码。

In [ ]:
PYTHON_PACKAGE_APPLICATION_DIR = "trainer"

! mkdir -p $PYTHON_PACKAGE_APPLICATION_DIR

#### 准备训练脚本

您的训练代码必须配置为将 TensorBoard 日志写入存储桶，其位置由 Vertex AI Training 通过预定义的环境变量 `AIP_TENSORBOARD_LOG_DIR` 自动提供。

通常可以通过将 `os.environ['AIP_TENSORBOARD_LOG_DIR']` 作为日志目录提供给开源 TensorBoard 日志写入 API 来实现。

例如，在 TensorFlow 2.x 中，您可以使用以下代码创建一个 tensorboard_callback：

    tensorboard_callback = tf.keras.callbacks.TensorBoard( 
      log_dir=os.environ['AIP_TENSORBOARD_LOG_DIR'], 
      histogram_freq=1) 
`AIP_TENSORBOARD_LOG_DIR` 是您创建自定义训练作业时提供的 `BASE_OUTPUT_DIR` 中的一部分。

要为您的训练作业启用 Vertex AI TensorBoard 分析程序，请将以下内容添加到您的训练脚本中：

在您的顶层导入中添加 cloud_profiler 导入：

    from google.cloud.aiplatform.training_utils import cloud_profiler

通过添加以下代码初始化 cloud_profiler 插件：

    cloud_profiler.init()

In [ ]:
%%writefile trainer/task.py

import tensorflow as tf
import argparse
import os
import sys, traceback
from google.cloud.aiplatform.training_utils import cloud_profiler

"""Train an mnist model and use cloud_profiler for profiling."""

def _create_model():
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10),
        ]
    )
    return model


def main(args):
    # Initialize the profiler.
    print('Initialize the profiler ...')
        
    try:
        cloud_profiler.init()
    except:
        ex_type, ex_value, ex_traceback = sys.exc_info()
        print("*** Unexpected:", ex_type.__name__, ex_value)
        traceback.print_tb(ex_traceback, limit=10, file=sys.stdout)
    
    print('The profiler initiated.')
    
    print('Loading and preprocessing data ...')
    mnist = tf.keras.datasets.mnist

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    print('Creating and training model ...')

    model = _create_model()
    model.compile(
      optimizer="adam",
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      metrics=["accuracy"],
    )

    log_dir = "logs"
    if 'AIP_TENSORBOARD_LOG_DIR' in os.environ:
      log_dir = os.environ['AIP_TENSORBOARD_LOG_DIR']

    print('Setting up the TensorBoard callback ...')
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=1)

    print('Training model ...')
    model.fit(
        x_train,
        y_train,
        epochs=args.epochs,
        verbose=0,
        callbacks=[tensorboard_callback],
    )
    print('Training completed.')

    print('Saving model ...')

    model_dir = "model"
    if 'AIP_MODEL_DIR' in os.environ:
      model_dir = os.environ['AIP_MODEL_DIR']
    tf.saved_model.save(model, model_dir)

    print('Model saved at ' + model_dir)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--epochs", type=int, default=100, help="Number of epochs to run model."
    )
    
    args = parser.parse_args()
    main(args)

准备Dockerfile.

In [ ]:
%%writefile Dockerfile
# Specifies base image and tag
FROM us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-9:latest
WORKDIR /root

# Installs additional packages as you need.
RUN pip3 install "google-cloud-aiplatform[cloud_profiler]>=1.20.0"
RUN pip3 install "protobuf==3.20.3"

# Copies the trainer code to the docker image.
RUN mkdir /root/trainer
COPY trainer/task.py /root/trainer/task.py

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

构建一个定制的容器映像并推送到您的私人Docker仓库。

In [ ]:
IMAGE_NAME = "tensorboard-custom-container"
IMAGE_URI = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{DOCKER_REPOSITORY}/{IMAGE_NAME}"

! gcloud builds submit --project {PROJECT_ID} --region={REGION} --tag {IMAGE_URI} --timeout=3600s --quiet

### 创建并运行自定义训练作业

使用自定义容器镜像配置一个[自定义作业](https://cloud.google.com/vertex-ai/docs/training/create-custom-job)。

In [ ]:
JOB_NAME = "tensorboard-job-unique"

job = aiplatform.CustomContainerTrainingJob(
    display_name=JOB_NAME, container_uri=IMAGE_URI
)

运行自定义训练任务

接下来，通过调用“run”方法并使用以下参数来运行自定义任务，以开始训练任务：

- `args`：要传递给训练脚本的命令行参数。
   - `--epochs`：训练的时期数。
- `replica_count`：用于训练的计算实例数量（replica_count = 1 表示单节点训练）。
- `machine_type`：用于计算实例的机器类型。
- `tensorboard`：TensorBoard实例。
- `service_account`：服务账号。
- `sync`：是否阻塞直到作业完成。

In [ ]:
base_output_dir = "{}/{}".format(BUCKET_URI, JOB_NAME)
MACHINE_TYPE = "n1-standard-4"
EPOCHS = 2
training_args = [
    "--epochs=" + str(EPOCHS),
]

job.run(
    args=training_args,
    replica_count=1,
    machine_type=MACHINE_TYPE,
    base_output_dir=base_output_dir,
    tensorboard=TENSORBOARD_INSTANCE_NAME,
    service_account=SERVICE_ACCOUNT,
)

查看TensorBoard分析仪仪表板

当自定义作业状态切换到运行时，您可以通过Google Cloud控制台上的自定义作业页或实验页访问Vertex AI TensorBoard分析仪仪表板。

Google Cloud指南[使用分析仪剖析模型训练性能](https://cloud.google.com/vertex-ai/docs/training/tensorboard-profiler)提供了详细的说明，以便访问Vertex AI TensorBoard分析仪仪表板并捕获性能分析会话。

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- Docker存储库
- 训练作业
- TensorBoard实例
- 云存储存储桶

In [ ]:
delete_tensorboard = True
delete_bucket = False

# Delete docker repository.
! gcloud artifacts repositories delete $DOCKER_REPOSITORY --project {PROJECT_ID} --location {REGION} --quiet

try:
    job.delete()
except Exception as e:
    print(e)

if delete_tensorboard:
    tensorboard.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI